In [17]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import math
import cv2
import lpips
from scipy import linalg
import skimage

from models.inception import InceptionV3
from torch.nn.functional import adaptive_avg_pool2d

In [18]:
data = torch.load('./datasets_unzipped/val.pt')
min_val = data.min()
max_val = data.max()
data = (data-min_val)/(max_val-min_val)

### LPIPS

In [19]:
loss_fn_alex = lpips.LPIPS(net='alex')

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/dhein/anaconda3/envs/main/lib/python3.7/site-packages/lpips/weights/v0.1/alex.pth


In [20]:
1-loss_fn_alex(data[0,0,:,:]*2-1,data[0,1,:,:]*2-1).item()

0.8218957781791687

### PSNR

In [66]:
def calculate_psnr(img1, img2):
    # img1 and img2 have range [0, 1]
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return float('inf')
    return 20 * math.log10(1.0 / math.sqrt(mse))

In [67]:
calculate_psnr(data[0,0,:,:].numpy(),data[0,1,:,:].numpy())

38.49900437946893

In [68]:
lpips.psnr(data[0,0,:,:].numpy(),data[0,1,:,:].numpy(),peak=1.)

38.49900422420026

### SSIM

In [78]:
def ssim(img1, img2):
    C1 = (0.01 * 1)**2
    C2 = (0.03 * 1)**2

    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    kernel = cv2.getGaussianKernel(11, 1.5)
    window = np.outer(kernel, kernel.transpose())

    mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]  
    mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
    mu1_sq = mu1**2
    mu2_sq = mu2**2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
    sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
    sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

    ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))
    return ssim_map.mean()


def calculate_ssim(img1, img2):
    '''calculate SSIM
    the same outputs as MATLAB's
    img1, img2: [0, 1]
    '''
    if not img1.shape == img2.shape:
        raise ValueError('Input images must have the same dimensions.')
    if img1.ndim == 2:
        return ssim(img1, img2)
    elif img1.ndim == 3:
        if img1.shape[2] == 3:
            ssims = []
            for i in range(3):
                ssims.append(ssim(img1[:,:,i], img2[:,:,i]))
            return np.array(ssims).mean()
        elif img1.shape[2] == 1:
            return ssim(np.squeeze(img1), np.squeeze(img2))
    else:
        raise ValueError('Wrong input image dimensions.')

In [79]:
calculate_ssim(data[0,0,:,:].numpy(),data[0,1,:,:].numpy())

0.8996802007899228